## Imports

In [1]:
import sys
from pathlib import Path
import torch
import gc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the path to the "chronos-forecasting" directory
sys.path.append("chronos-forecasting")

# import the basepipeline from the folder chronos-forecasting/src/chronos/base.py not from the installed package

from src.chronos.chronos import ChronosPipeline, ForecastType

## Global Params

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOKENIZER_CLASS = "IQRScaleUniformBins"
MODEL_ID = "/teamspace/studios/this_studio/chronoslocal/chronos-forecasting/scripts/output/run-5/checkpoint-100000"
MODEL_SIZE= "tiny" # or use "MeanScaleUniformBins" or "LogScaleUniformBins"

## Load the model

In [3]:
pipeline = ChronosPipeline.from_pretrained(
    MODEL_ID,  
    device_map=DEVICE,
    torch_dtype=torch.bfloat16,
    tokenizer_class=TOKENIZER_CLASS,
)

Using IQR Scaling
T-Config: ChronosConfig(tokenizer_class='IQRScaleUniformBins', tokenizer_kwargs={'low_limit': -15.0, 'high_limit': 15.0}, context_length=512, prediction_length=64, n_tokens=4096, n_special_tokens=2, pad_token_id=0, eos_token_id=1, use_eos_token=True, model_type='seq2seq', num_samples=20, temperature=1.0, top_k=50, top_p=1.0)


## Helper Functions for the evaluation


In [4]:

from typing import Iterable
import datasets
from gluonts.dataset.split import split
from gluonts.ev.metrics import MASE, MeanWeightedSumQuantileLoss
from gluonts.itertools import batcher
from gluonts.model.evaluation import evaluate_forecasts
from gluonts.model.forecast import QuantileForecast, SampleForecast
from tqdm.auto import tqdm
import numpy as np

def to_gluonts_univariate(hf_dataset: datasets.Dataset):
    series_fields = [
        col
        for col in hf_dataset.features
        if isinstance(hf_dataset.features[col], datasets.Sequence)
    ]
    series_fields.remove("timestamp")
    dataset_length = hf_dataset.info.splits["train"].num_examples * len(series_fields)

    # Assumes that all time series in the dataset have the same frequency
    dataset_freq = pd.DatetimeIndex(hf_dataset[0]["timestamp"]).to_period()[0].freqstr

    gts_dataset = []
    for hf_entry in hf_dataset:
        for field in series_fields:
            gts_dataset.append(
                {
                    "start": pd.Period(
                        hf_entry["timestamp"][0],
                        freq=dataset_freq,
                    ),
                    "target": hf_entry[field],
                }
            )
    assert len(gts_dataset) == dataset_length

    return gts_dataset


def generate_forecasts(data: Iterable, pipeline: ChronosPipeline, prediction_length: int, batch_size: int):
    forecast_outputs = []
    for batch in tqdm(batcher(data, batch_size=batch_size)):
        context = [torch.tensor(entry[0]["target"], dtype=torch.bfloat16) for entry in batch]
        forecast_outputs.append(
            pipeline.predict(
                context,
                prediction_length=prediction_length,
            ).numpy()
        )
    forecast_outputs = np.concatenate(forecast_outputs)

    # Convert forecast samples into gluonts Forecast objects
    forecasts = []
    for item, ts in zip(forecast_outputs, data):
        forecast_start_date = ts[0]["start"] + len(ts[0]["target"])

        if pipeline.forecast_type == ForecastType.SAMPLES:
            forecasts.append(
                SampleForecast(samples=item, start_date=forecast_start_date)
            )
        elif pipeline.forecast_type == ForecastType.QUANTILES:
            forecasts.append(
                QuantileForecast(
                    forecast_arrays=item,
                    forecast_keys=list(map(str, pipeline.quantiles)),
                    start_date=forecast_start_date,
                )
            )

    return forecasts





/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [7]:
import yaml


dataset_name = "monash_electricity_hourly"
prediction_length = 24
offset = -24
num_rolls = 1

with open("scripts/evaluation/configs/in-domain.yaml") as fp:
    dataset_yaml = yaml.safe_load(fp)

result_rows = []

for config in dataset_yaml:
    dataset_name = config["name"]
    prediction_length = config["prediction_length"]
    offset = config["offset"]
    num_rolls = config["num_rolls"]


    ds = datasets.load_dataset("autogluon/chronos_datasets", dataset_name, split="train")
    ds.set_format("numpy")  # sequences returned as numpy arrays

    gts_dataset = to_gluonts_univariate(ds)

    _, test_template = split(gts_dataset, offset=offset)
    test_data = test_template.generate_instances(prediction_length, windows=num_rolls)

    print(
            f"Generating forecasts for {dataset_name} "
            f"({len(test_data.input)} time series)"
        )


    forecasts = generate_forecasts(test_data, pipeline, prediction_length, batch_size=32)

    print(f"Evaluating forecasts for {dataset_name}")


    metrics = (
            evaluate_forecasts(
                forecasts,
                test_data=test_data,
                metrics=[
                    MASE(),
                    MeanWeightedSumQuantileLoss(np.arange(0.1, 1.0, 0.1)),
                ],
                batch_size=5000,
            )
            .reset_index(drop=True)
            .to_dict(orient="records")
        )
    results = {"dataset": dataset_name, "model": MODEL_SIZE, **metrics[0]}
    result_rows.append(results)

results_df = (
        pd.DataFrame(result_rows)
        .rename(
            {"MASE[0.5]": "MASE", "mean_weighted_sum_quantile_loss": "WQL"},
            axis="columns",
        )
)
results_df.to_csv(f"results_{TOKENIZER_CLASS}_in_domain_full_train_model_{MODEL_SIZE}.csv", index=False)




Generating forecasts for electricity_15min (370 time series)


0it [00:00, ?it/s]